<a href="https://colab.research.google.com/github/Leontiy2/MachineLearningPRO/blob/main/ProjectHuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import userdata
Hf = userdata.get("Hf")

from huggingface_hub import login
login(Hf)

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import Dataset
# from peft

data = [
    {"prompt": "Що таке сила тяжіння?", "response": "Сила тяжіння - це взаємне притягування сил з масою..."},
    {"prompt": "Поясни, що таке сила виштовхування? (закон Архімеда)", "response": "Сила, що спрямоване на тіло, заурене у рідину, напрямлена у гору."},
    {"prompt": "Різниця між кінетичною та потенціальною енергією?", "response": "Кінетична - це сила руху, а потенціальна - сила збережена в об'єкті."},
    {"prompt": "Що таке DB?", "response": "DB - база даних."},
    {"prompt": "Що таке математична статистика?", "response": "Наука, що досліджує дані, отримані у результаті дослідження даних про певний об'єкт."},
    {"prompt": "Що таке сигнатура функції?", "response": "Ім'я та типи параметрів функції."},
    {"prompt": "Яка різниця між ssd та hdd дисками", "response": "SSD - це швидший, безшумний і надійніший накопичувач без рухомих частин, тоді як HDD повільніший, але дешевший і з більшим обсягом пам’яті."},
    {"prompt": "Скількі води потрібно пити людині в день", "response": "Людині потрібно пити близько 1,5–2 літрів води на день."},
    {"prompt": "What is an apple", "response": "An apple is a fruit"},
    {"prompt": "", "response": ""}
]

for item in data:
  item['text'] = item['prompt'].strip() + '\nAnswer: ' + item['response'].strip()



dataset = Dataset.from_list([{"text": d["text"]} for d in data])

model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
def tokenize_fn(examples):
  tokens = tokenizer(examples['text'], truncation=True, padding='max_length', max_length = 128)
  tokens['labels'] = tokens["input_ids"].copy()
  return tokens


tokenized = dataset.map(tokenize_fn, batched = True, remove_columns = ['text'])

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Налаштування LoRA

In [4]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

lora_config = LoraConfig(
    r = 8,
    lora_alpha = 32,
    target_modules = ["c_attn", "q_proj", "v_proj"],
    lora_dropout = 0.1,
    bias = "none",
    task_type = "CAUSAL_LM"
)

model = prepare_model_for_kbit_training(model)

model = get_peft_model(model, lora_config)

/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/layer.py:2285: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


Параметри тренування і запуску

In [5]:
training_args = TrainingArguments(
    output_dir = "./lora-distilgpt2",
    per_device_train_batch_size = 4,
    num_train_epochs = 3,
    logging_steps = 10,
    save_total_limit = 1,
    fp16 = False,
    # report_to = "none"  # "wandb"
    # hub_model_id = ""
    hub_token = Hf
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized,
)

trainer.train()

wandb: Currently logged in as: leone-rust727 (leone-rust727-none) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=9, training_loss=5.222100575764974, metrics={'train_runtime': 50.1758, 'train_samples_per_second': 0.598, 'train_steps_per_second': 0.179, 'total_flos': 983260200960.0, 'train_loss': 5.222100575764974, 'epoch': 3.0})

Збереження адаптера і токенізатора

In [6]:
model.save_pretrained("./lora-distilgpt2-adapter")
tokenizer.save_pretrained("./lora-distilgpt2-adapter")

('./lora-distilgpt2-adapter/tokenizer_config.json',
 './lora-distilgpt2-adapter/special_tokens_map.json',
 './lora-distilgpt2-adapter/vocab.json',
 './lora-distilgpt2-adapter/merges.txt',
 './lora-distilgpt2-adapter/added_tokens.json',
 './lora-distilgpt2-adapter/tokenizer.json')

Генерація тексту (інференс) після адаптації

In [7]:
from peft import PeftModel
base_model = AutoModelForCausalLM.from_pretrained("distilgpt2")
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token
model = PeftModel.from_pretrained(base_model, "./lora-distilgpt2-adapter")

model.eval()

# prompt = "Що таке сила тяжіння?\nAnswer:"
prompt = "What is an apple?\nAnswer:"
inputs = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(
    **inputs,
    max_length=150,
    do_sample = True,
    top_p = 0.9,
    temperature = 0.7,
    pad_token_id = tokenizer.eos_token_id
    )
print(tokenizer.decode(outputs[0], skip_special_tokens = True))

What is an apple?
Answer: It is a apple.
Answer: It is a tree.
Answer: It is a tree.
Answer: It is a tree.
Answer: It is a tree.
Answer: It is a tree.
Answer: It is a tree.
Answer: It is a tree.
Answer: It is a tree.
Answer: It is a tree.
Answer: It is a tree.
Answer: It is a tree.
Answer: It is a tree.
Answer: It is a tree.
Answer: It is a tree.
Answer: It is a tree.
Answer: It is a tree.
Answer: It is a tree.

